In [8]:
from sklearn import tree
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import zipfile
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.preprocessing import RobustScaler, StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_absolute_error

from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMRegressor, LGBMClassifier

In [2]:
zf = zipfile.ZipFile('C:/Users/Serueja/Downloads/spaceship-titanic.zip')
test = pd.read_csv(zf.open('test.csv'))
train = pd.read_csv(zf.open('train.csv'))
sample_submission = pd.read_csv(zf.open('sample_submission.csv'))
# train.drop(['Id'], axis=1, inplace=True)
# test.drop(['Id'], axis=1, inplace=True)
X = train.drop(['Transported'], axis=1)
y = train['Transported']

In [3]:
train_features = train.drop(['Transported'], axis=1)
test_features = test
features = pd.concat([train_features, test_features]).reset_index(drop=True)
features.drop(['Name'], axis=1, inplace=True)
features['PassengerId'] = features['PassengerId'].apply(str)
features['PassengerId'].dtype
new_df = features['PassengerId'].str.split('_',expand=True)
new_df.columns = ['GroupId', 'PersonalId']
features = pd.concat([features, new_df], axis=1, join='inner')

features.drop(['PassengerId'], axis = 1, inplace=True)
features.HomePlanet.fillna('Earth', inplace=True)
features.CryoSleep.fillna('False', inplace=True)
features.Destination.fillna('TRAPPIST-1e', inplace=True)
features.Age.fillna(27, inplace=True)
features.RoomService.fillna(0, inplace=True)
features.FoodCourt.fillna(0, inplace=True)
features.ShoppingMall.fillna(0, inplace=True)
features.Spa.fillna(0, inplace=True)
features.VRDeck.fillna(0, inplace=True)
features.drop(['Cabin'], axis =1, inplace=True)
dummies_home = pd.get_dummies(features['HomePlanet'], prefix='Home')
dummies_dest = pd.get_dummies(features['Destination'], prefix='Dest')

features = pd.concat([features, dummies_home, dummies_dest], axis=1, join='inner')
features.drop(['HomePlanet', 'Destination'], axis=1, inplace=True)
features['CryoSleep'] = features['CryoSleep'].transform(lambda x: 1 if x == True else 0)
features['VIP'] = features['VIP'].transform(lambda x: 1 if x == True else 0)
features['TotalSpent'] = features['RoomService'] + features['FoodCourt'] + features['ShoppingMall'] + features['Spa'] + features['VRDeck']

In [4]:
print(features.CryoSleep.isna().any())
print(features.Age.isna().any())
print(features.RoomService.isna().any())
print(features.FoodCourt.isna().any())
print(features.ShoppingMall.isna().any())
print(features.Spa.isna().any())
print(features.VRDeck.isna().any())

False
False
False
False
False
False
False


In [5]:
features

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,GroupId,PersonalId,Home_Earth,Home_Europa,Home_Mars,Dest_55 Cancri e,Dest_PSO J318.5-22,Dest_TRAPPIST-1e,TotalSpent
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0001,01,0,1,0,0,0,1,0.0
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,0002,01,1,0,0,0,0,1,736.0
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0003,01,0,1,0,0,0,1,10383.0
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0003,02,0,1,0,0,0,1,5176.0
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,0004,01,1,0,0,0,0,1,1091.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,1,34.0,0,0.0,0.0,0.0,0.0,0.0,9266,02,1,0,0,0,0,1,0.0
12966,0,42.0,0,0.0,847.0,17.0,10.0,144.0,9269,01,1,0,0,0,0,1,1018.0
12967,1,27.0,0,0.0,0.0,0.0,0.0,0.0,9271,01,0,0,1,1,0,0,0.0
12968,0,27.0,0,0.0,2680.0,0.0,0.0,523.0,9273,01,0,1,0,0,0,1,3203.0


In [6]:
grid_params = {
    'n_estimators': [100, 150],
    'max_depth': [2, 4, 8, 10],
    'max_features': [8, 10, 12],
    'min_samples_leaf': [1, 2, 4]
}
X = features.iloc[:len(y), :]
X_sub = features.iloc[len(y):, :]
X.shape, y.shape, X_sub.shape

((8693, 17), (8693,), (4277, 17))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)

Следующая ячейки будет сугубо для XGBClassifier

In [18]:
X_train_xgb = X_train.drop(['GroupId', 'PersonalId'], axis = 1)
X_test_xgb = X_test.drop(['GroupId', 'PersonalId'], axis = 1)
X_sub_xgb = X_sub.drop(['GroupId', 'PersonalId'], axis = 1)

In [12]:
xgb_model_clf = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)
parameters_for_xgb_clf = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}
grid_search_for_xgb_model_clf = GridSearchCV(
    estimator=xgb_model_clf,
    param_grid=parameters_for_xgb_clf,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)
grid_search_for_xgb_model_clf.fit(X_train_xgb, y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_c...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None, ...),
             n_jobs=10,
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(2, 10),
                         'n_estimators': range(60, 220, 40)},
             scoring='roc_auc', verbose=True)

In [13]:
xgb_model_clf_best_estimator=grid_search_for_xgb_model_clf.best_estimator_
xgb_model_clf_best_estimator.score(X_test_xgb, y_test)

0.7926106657371906

Следующая ячейка будет сугубо для RandomForestClassifier

In [35]:
rf_model = RandomForestClassifier(random_state=50)
rf_model.fit(X_train, y_train)
print('The accuracy of untuned RFClf is ', cross_val_score(rf_model, X, y, cv=5).mean())

The accuracy of untuned RFClf is  0.7247271853789494


In [36]:
rf_model_for_cv = RandomForestClassifier(random_state=50)
CV_rfc = GridSearchCV(estimator=rf_model_for_cv, param_grid= grid_params, cv = 5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_score_)
print(CV_rfc.best_params_)
best_clf_rf = CV_rfc.best_estimator_
best_clf_rf.score(X_test, y_test)

In [40]:
rfr_predict = pd.DataFrame({'PassengerId':test['PassengerId'],'Transported':pd.Series(best_clf_rf.predict(X_sub))})

In [21]:
xgb_predict = pd.DataFrame({'PassengerId':test['PassengerId'],'Transported':pd.Series(xgb_model_clf_best_estimator.predict(X_sub_xgb))})

In [33]:
xgb_predict.to_csv('xgb_clf_grid_search_submission.csv', index=False)

In [31]:
xgb_predict['Transported'] = xgb_predict['Transported'].astype(bool)

In [32]:
xgb_predict

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [41]:
rfr_predict.to_csv('rf_clf_grid_search_submission.csv', index=False)